Este archivo genera un split del dataset de imagenes y sus correspondientes asignaciones en csv. Genera

In [ ]:
import os
import sys
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv

In [ ]:
RANDOM_STATE = 42
CWD = os.getcwd()

DATA_PATH = os.path.join(CWD, "dataset")

In [ ]:
def split_train_test():

    try:
        os.mkdir(path=os.path.join(DATA_PATH, "train"))
        os.mkdir(path=os.path.join(DATA_PATH, "test"))
        os.mkdir(path=os.path.join(DATA_PATH, "train/images"))
        os.mkdir(path=os.path.join(DATA_PATH, "test/images"))
    except FileExistsError:
        pass

    df_ISIC = pd.read_csv("dataset/ISIC_2019_Training_GroundTruth.csv")
    df_ISIC = df_ISIC.drop('UNK', axis = 1)

    y_train = pd.DataFrame()
    y_test = pd.DataFrame()

    clases = df_ISIC.columns[1:]
    
    for clase in clases:
        print(clase)
        clase_110 = df_ISIC[df_ISIC[clase] == 1].sample(110, random_state=RANDOM_STATE)
        y_train = pd.concat([y_train, clase_110.head(100)])
        y_test = pd.concat([y_test, clase_110.tail(10)])

    # Generar csv de train y de test
    y_train.to_csv("dataset/train/y_train.csv", index=False)
    y_test.to_csv("dataset/test/y_test.csv", index=False)

    y_train_imgs_names = y_train['images']
    y_test_imgs_names = y_test['images']

    
    img = cv2.imread("path")
    img = cv2.

    
    return y_train, y_test

y_train, y_test = split_train_test()



MEL
NV
BCC
AK
BKL
DF
VASC
SCC
